In [28]:
from PIL import Image
import pytesseract
import os
import cv2
import numpy as np

# Path to the Tesseract executable (update if necessary, the path is typically /usr/local/bin/tesseract)
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'

def extract_alphabets(image_path):
    # Open the image using Pillow (PIL)
    image = cv2.imread(image_path)

    # Use pytesseract to extract text from the image
    # extracted_text = pytesseract.image_to_string(image, config='--psm 6')

    # Filter alphabets from the extracted text
    # alphabets = ''.join(char for char in extracted_text if char.isalpha())


    extracted_boxes = pytesseract.image_to_boxes(image, config='--psm 6')
    img = image.copy()
    

    for box in extracted_boxes.splitlines():
        b = box.split()
        x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
        cv2.rectangle(img, (x, y), (w, h), (0, 255, 0), 2)

    cv2.imwrite(os.path.join(os.getcwd(), 'boundingBoxesTesseract/bounding_box1_sample1.png'),img)

    return extracted_boxes

# Replace 'your_image_path.png' with the path to your image
image_path = os.path.join(os.getcwd(), 'samples/sample2.png')
print(image_path)
result = extract_alphabets(image_path)

print(f"Extracted Alphabets: {result}")

/Users/boost/Documents/deep_learning/project/samples/sample2.png


[ WARN:0@2244.849] global loadsave.cpp:248 findDecoder imread_('/Users/boost/Documents/deep_learning/project/samples/sample2.png'): can't open/read file: check file path/integrity


TypeError: Unsupported image object